## installa ambiente

In [ ]:
!pip install gdal
!pip install geopandas
!pip install vtk

### Elabora AOI

In [ ]:
geojson = '''
{
  "type": "Feature",
  "properties": {},
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [-118.4711, 34.3916],
        [-118.47, 34.3915],
        [-118.4688, 34.3916],
        [-118.467, 34.3921],
        [-118.4658, 34.3924],
        [-118.4646, 34.3928],
        [-118.4629, 34.3936],
        [-118.461, 34.3942],
        [-118.4578, 34.3939],
        [-118.4595, 34.399],
        [-118.46, 34.3994],
        [-118.4631, 34.3976],
        [-118.4674, 34.3952],
        [-118.469, 34.394],
        [-118.4699, 34.3934],
        [-118.471, 34.3926],
        [-118.4717, 34.3918],
        [-118.4711, 34.3916]
      ]
    ]
  }
}
'''
AOI = gpd.GeoDataFrame.from_features([json.loads(geojson)])
BUFFER = 0.025
# geometry is too small for the processing, enlarge it
AOI['geometry'] = AOI.buffer(BUFFER)

### Elabora file output

In [ ]:
print(velocity_ps.shape)
#print(velocity_ps[:10])
print(zmin)
print(zmax)

plt.figure(figsize=(12, 4), dpi=300)

# subsidence point from
geojson = '''
{
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [-118.4629,34.3946]
  },
  "properties": {}
}
'''
POI = gpd.GeoDataFrame.from_features([json.loads(geojson)])

velocity_sbas_ll = sbas.ra2ll(velocity_sbas)
velocity_ps_ll = sbas.ra2ll(velocity_ps)
points_sbas = sbas.as_geo(sbas.ra2ll(velocity_sbas))#.rio.clip(AOI.geometry)
points_ps = sbas.as_geo(sbas.ra2ll(velocity_ps))#.rio.clip(AOI.geometry)

print("Dimensione massima:", points_sbas.size)

max_lat = points_sbas['lat'].max()
min_lat = points_sbas['lat'].min()

max_lon = points_sbas['lon'].max()
min_lon = points_sbas['lon'].min()
print("Coordinate massime e minime:")
print("Latitudine massima:", max_lat.values)
print("Latitudine minima:", min_lat.values)
print("Longitudine massima:", max_lon.values)
print("Longitudine minima:", min_lon.values)

print(velocity_sbas_ll['lat'][0].values)


velocity_sbas_ll.to_dataframe().to_csv('/content/velocity_sbas_ll.csv')
velocity_ps_ll.to_dataframe().to_csv('/content/velocity_ps_ll.csv')
points_sbas.to_dataframe().to_csv('/content/points_sbas.csv')
points_ps.to_dataframe().to_csv('/content/points_ps.csv')
#velocity_sbas_ll.to_netcdf('/content/velocity_sbas_ll.nc')
#velocity_ps_ll.to_netcdf('/content/velocity_ps_ll.nc')

x, y = [(geom.x.item(), geom.y.item()) for geom in sbas.geocode(POI).geometry][0]
print("Punto di interesse x={} e y={}".format(x, y))
disp_pixel = disp_ps.sel(y=y, x=x, method='nearest')
stl_pixel = stl_ps.sel(y=y, x=x, method='nearest')
#plt.plot(disp_pixel.date, disp_pixel, c='r', lw=2, label='Displacement POI')

import pandas as pd

# Creazione di un DataFrame pandas
data = {'Data': disp_pixel.date, 'Displacement': disp_pixel}
df = pd.DataFrame(data)

print("Punto 0 Data={}; Displacement={}".format(df['Data'].iloc[0], df['Displacement'].iloc[0]))

# Stampa del DataFrame
#print(df)


for lat, lon in zip(velocity_ps_ll.coords['lat'].values, velocity_ps_ll.coords['lon'].values):
    valore = velocity_ps_ll.sel(lat=lat, lon=lon).values
    print("Punto di interesse lat={} e lon={}; valore={}".format(lat, lon, valore))
    exit


# Converti l'oggetto xarray.DataArray in un DataFrame pandas con coordinate separate
df_ll = velocity_ps_ll.reset_coords(drop=True).to_dataframe(name='valore')

# Visualizza il DataFrame
print(df_ll)

### trasforma 'grd' files

In [ ]:
import platform, sys, os
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
import json
from dask.distributed import Client
import dask
import warnings
warnings.filterwarnings('ignore')

from osgeo import gdal

# Percorso del file GRD di input
input_grd = "disp_ps.grd"#"/content/raw_golden_desc/disp_ps.grd"
if os.path.exists(input_grd):
    print("Il file esiste.")
else:
    print("Il file non esiste.")
    
# Percorso del file XYZ di output
output_xyz = "disp_ps.xyz"#"/content/raw_golden_desc/disp_ps.xyz"

# Apri il file GRD
dataset = gdal.Open(input_grd)

# Leggi i dati del raster
data = dataset.ReadAsArray()

# Ottieni le informazioni sulle dimensioni del raster
cols = dataset.RasterXSize
rows = dataset.RasterYSize
print("Dimesioni raster rows={} e cols={}".format(rows, cols))

# Ottieni le informazioni sulla trasformazione geospaziale
geotransform = dataset.GetGeoTransform()

# Calcola l'estensione spaziale del raster
x_min = geotransform[0]
y_max = geotransform[3]
x_max = x_min + geotransform[1] * cols
y_min = y_max + geotransform[5] * rows

# Calcola la risoluzione del raster
x_resolution = geotransform[1]
y_resolution = geotransform[5]

# Scrivi i dati in formato XYZ
with open(output_xyz, "w") as outfile:
    for y in range(rows):
        for x in range(cols):
            # Calcola le coordinate del punto
            x_coord = x_min + (x + 0.5) * x_resolution
            y_coord = y_max - (y + 0.5) * y_resolution
            #print("Coordinate x_coord={} e y_coord={}".format(x_coord, y_coord))
            # Verifica che gli indici siano all'interno dei limiti dell'array
            if 0 <= y < len(data) and 0 <= x < len(data[y]):
                if not np.all(np.isnan(data[y][x])):
                    # Scrivi le coordinate e il valore del punto nel file XYZ
                    shape = data[y][x].shape
                    #print("Punto di interesse x={} e y={} Dimensioni di data[y][x]: {}".format(x, y, shape))
                    outfile.write(f"{x_coord} {y_coord} {data[y][x]}\n")
                else:
                    #print(f"Valore NaN trovato: y={y}, x={x}")
                    a = 1
            else:
                # Se gli indici sono al di fuori dei limiti, stampa un messaggio di avviso
                b = 1
                #print(f"Indici fuori dai limiti: y={y}, x={x}")

print(f"Terminato")

### legge 'vtk' files

In [ ]:
import platform, sys, os

import vtk

# Percorso del file VTK
file_vtk = 'velocity_ps.vtk'

if os.path.exists(file_vtk):
    print("Il file esiste.")
else:
    print("Il file non esiste.")
    
# Coordinata (x, y) del punto di interesse
x = 1  # Coordinata x del punto di interesse
y = 1  # Coordinata y del punto di interesse

# Crea un lettore VTK per il file
reader = vtk.vtkStructuredGridReader()
reader.SetFileName(file_vtk)
reader.Update()

# Ottieni l'output del lettore
output = reader.GetOutput()

# Ottenere il numero di punti nel grid
num_points = output.GetNumberOfPoints()
print("num_points={}".format(num_points))

# Cerca il punto di interesse
point_id = -1
for i in range(num_points):
    point = output.GetPoint(i)
    print("Valore della coordinata x={}, y={}, z={}".format(point[0], point[1], point[2]))
    if point[0] == x and point[1] == y:
        point_id = i
        break

# Se il punto di interesse è stato trovato
if point_id != -1:
    # Ottieni il valore della coordinata z del punto di interesse
    z_value = output.GetPoint(point_id)[2]
    print("Valore della coordinata z per x={}, y={}: {}".format(x, y, z_value))
else:
    print("Punto di interesse non trovato per x={} e y={}".format(x, y))